In [19]:
import nltk
from nltk.corpus import twitter_samples
import random
import numpy as np
import pandas as pd

In [20]:
pos_tweets = open("tweets.json","r")
positive_tweets_list = []
for tweet in pos_tweets:
    positive_tweets_list.append(tweet)
print(positive_tweets_list)

['  \n', "ï»¿it's extremely effective . \n", 'offers that rare combination of entertainment and education . \n', 'a masterpiece four years in the making . \n', "the movie's ripe , enrapturing beauty will tempt those willing to probe its inscrutable mysteries . \n", 'offers a breath of the fresh air of true sophistication .\n', 'funny but perilously slight . \n', "overall very good for what it's trying to do . \n", "it's a charming and often affecting journey . \n", 'psychologically savvy . \n', 'for movie lovers as well as opera lovers , tosca is a real treat . \n', 'unflinchingly bleak and desperate\n', 'one from the heart . \n', "more concerned with sade's ideas than with his actions . the movie achieves as great an impact by keeping these thoughts hidden as . . . [quills] did by showing them . \n", 'an entertaining , colorful , action-filled crime story with an intimate heart . \n', "while undisputed isn't exactly a high , it is a gripping , tidy little movie that takes mr . hill hi

In [21]:
neg_tweets = open("negative.json","r",encoding = 'utf-8')
negative_tweets_list = []
for tweet in neg_tweets:
    negative_tweets_list.append(tweet)
print(negative_tweets_list)

["\ufeffif sinise's character had a brain his ordeal would be over in five minutes but instead the plot goes out of its way to introduce obstacles for him to stumble over . \n", 'too slow for a younger crowd , too shallow for an older one . \n', 'there\'s a reason the studio didn\'t offer an advance screening . " the adventures of pluto nash " is a big time stinker . \n', 'a punch line without a premise , a joke built entirely from musty memories of half-dimensional characters . \n', "takes one character we don't like and another we don't believe , and puts them into a battle of wills that is impossible to care about and isn't very funny . \n", "the things this movie tries to get the audience to buy just won't fly with most intelligent viewers . \n", "even if the enticing prospect of a lot of nubile young actors in a film about campus depravity didn't fade amid the deliberate , tiresome ugliness , it would be rendered tedious by avary's failure to construct a story with even a trace of

In [29]:
print(len(negative_tweets_list))
print(type(negative_tweets_list))

223
<class 'list'>


In [24]:
print(len(positive_tweets_list))

271


In [34]:
nltk.download('twitter_samples')
all_pos_tweets = twitter_samples.strings('positive_tweets.json')
all_neg_tweets = twitter_samples.strings('negative_tweets.json')


[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\shobh\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [122]:
#print(all_pos_tweets[4000:])

In [56]:
test_pos = positive_tweets_list[:]
train_pos = all_pos_tweets[:3500]
test_neg = negative_tweets_list[:]
train_neg = all_neg_tweets[:3500]

test_data_x = test_pos + test_neg
train_data_x = train_pos + train_neg


In [47]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [48]:
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (7000, 1)
test_y.shape = (494, 1)


In [50]:
# Preprocessing the data 

In [78]:
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [52]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shobh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [67]:
train_data_clean = []
for tweet in train_data_x:
    tweet2 = re.sub(r'RT[\s]+','',tweet)
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)
    tweet2 = re.sub(r'#', '', tweet2)
    train_data_clean.append(tweet2)


In [69]:
test_data_clean = []
for tweet in test_data_x:
    tweet2 = re.sub(r'RT[\s]+','',tweet)
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)
    tweet2 = re.sub(r'#', '', tweet2)
    test_data_clean.append(tweet2)

In [74]:
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
test_tweet_token = []
for tweet in test_data_clean:
    tweet_token = tokenizer.tokenize(tweet)
    test_tweet_token.append(tweet_token)

In [121]:
#print(test_tweet_token)

In [76]:
print(train_data_x[0])

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [77]:
# writing a function to process the tweets :)

In [89]:
def process_tweets(tweet):
    
    stemmer = PorterStemmer() 
    stopwords_eng = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)  #removing the hyperlinks
    tweet = re.sub(r'#', '', tweet)
    
    tokenizer = TweetTokenizer(preserve_case = False, strip_handles = True,reduce_len = True) #tokenize tweet function
    
    tweet_tokens = tokenizer.tokenize(tweet)
    
    tweet_clean = []
    
    for word in tweet_tokens:
        
        if(word not in stopwords_eng and word not in string.punctuation): #removing stops words and punctuation
            
            stem_words = stemmer.stem(word)
            tweet_clean.append(stem_words)
            
    return tweet_clean
    

In [90]:
def building_freqs(tweets,ys):
    
    yslist = np.squeeze(ys).tolist()  #squeezing the array into a list
    
    freqs = {}
    for y,tweet in zip(yslist,tweets):
        for word in process_tweets(tweet):
            pair = (word,y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

In [91]:
freqs = building_freqs(train_data_x,train_y)

In [92]:
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 10350


In [94]:
print(process_tweets(train_data_x[0]))

['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [95]:
# here comes the logistic regression part -->(refrenced from google)

In [96]:
# sigmoid function

def sigmoid(z):
    
    h = 1/(1 + np.exp(-z))
    
    return h

In [97]:
# cost function and gradient function

def gradientDescent(x,y,theta,alpha,num_iters):
    
    m = x.shape[0]
    for i in range(0,num_iters):
        
        z = np.dot(x,theta)
        h = sigmoid(z)
        
        
        J = -1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h)))                                                    

       
        theta = theta - (alpha/m) * np.dot(x.transpose(),(h-y))
        
    J = float(J)
    return J, theta
        

In [98]:
# extracting features

def extract_features(tweet,freqs):
    
    word_l = process_tweets(tweet)
    x = np.zeros((1,3))
    
    x[0,0] = 1
    
    for word in word_l:
        
        x[0,1] += freqs.get((word,1.0),0)  # incrementing the word count for +ve word labels
        
        x[0,2] += freqs.get((word,0.0),0)  # incrementing the word count for -ve word labels
        
        
    assert(x.shape==(1,3))
    return x

In [99]:
#checking the function
tmp1 = extract_features(train_data_x[0], freqs)
print(tmp1)

[[1.000e+00 2.663e+03 5.500e+01]]


In [100]:
#training the model --> hope it works :)

In [102]:
X = np.zeros((len(train_data_x),3))
for i in range(len(train_data_x)):
    
    X[i,:] = extract_features(train_data_x[i],freqs)
    
Y = train_y

J,theta = gradientDescent(X,Y,np.zeros((3,1)),1e-9,1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")


The cost after training is 0.27438442.
The resulting vector of weights is [7e-08, 0.00050449, -0.00055522]


In [103]:
# the orcale(predicting function :))

In [105]:
def predict_tweet(tweet,freqs,theta):
    
    x = extract_features(tweet,freqs)
    
    y_pred = sigmoid(np.dot(x,theta))
    
    return y_pred

In [106]:
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

I am happy -> 0.514789
I am bad -> 0.494964
this movie should have been great. -> 0.512620
great -> 0.512810
great great -> 0.525604
great great great -> 0.538364
great great great great -> 0.551073


In [108]:
# performance check --> hope it is better than my college cgpa :)

In [116]:
def testing_logistic_regression(test_data_x,test_y,freqs,theta):
    
    y_hat = []
    
    for tweet in test_data_x:
        
        y_pred = predict_tweet(tweet,freqs,theta)
        if y_pred > 0.5:
            y_hat.append(1)  # positive tweet
        else:
            y_hat.append(0)  # negative tweet
            
    
    #accuracy of the model
    
    accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_data_x)
    
    return accuracy
    

In [119]:
tmp_accuracy = testing_logistic_regression(test_data_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.5466
